In [2]:
import numpy as np

step11: 可変長の引数（順伝播）

11.1 Functionクラスの修正

In [3]:
class Variable:

    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray): # dataがndarrayインスタンスではない時
                raise TypeError('{} is not supported'.format(type(data))) # 入力されたdataのデータ型を{}に表示

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None:
                funcs.append(x.creator)

In [4]:
def as_array(x):
    if np.isscalar(x): # もしfloatなどのスカラ系の型なら
        return np.array(x) # 改めて型をndarrayにする
    return x

In [5]:
class Function:
    def __call__(self, inputs):
        # 入力が複数であること想定してリストにする
        xs = [x.data for x in inputs] # 複数の入力Variableインスタンスinputsから一つずつxを取り出して、そのインスタンスのデータを新しいリストに格納
        ys = self.forward(xs) # この時点では出力ysはタプル
        # print('ys: ', ys)
        # ysタプル内から一つずつ要素yを取り出してndarrayに変換しVariableインスタンスのデータに格納、そのVariableインスタンスを新しいリストに格納する
        outputs = [Variable(as_array(y)) for y in ys] #要素が一つでもタプルで渡せば一回だけのループが可能

        for output in outputs:
            output.set_creator(self) # それぞれの出力Variableインスタンスに対して、出力した親（関数）を関連付ける（皆共通の関数）
        self.inputs = inputs
        self.outputs = outputs

        return outputs

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

11.2 Addクラスの実装

In [6]:
class Add(Function):
    def forward(self, xs):
        x0, x1 = xs
        y = x0 + x1
        return (y,) # この後ループで回すから要素が一つでもタプルにする必要がある(yだけを返すとこの後エラーになる)

In [7]:
xs = [Variable(np.array(2)), Variable(np.array(3))]
f = Add()
ys = f(xs) # 出力は要素一つだけのリスト
print(len(ys))
print(ys)
y = ys[0] # 出力データを見るにはわざわざ出力リストから最初の要素を指定しなければらなない
print(y.data)

1
5


step12: 可変長の引数（改善編）：　

12.1 １つ目の改善：関数を使いやすく（関数の引数にリストを必要としない仕様にする）

In [8]:
class Function:
    def __call__(self, *inputs): # アスタリスクをつけることで任意の数の引数（変長引数）を受け付ける(タプル)＝入力リストを作成しなくていい
        xs = [x.data for x in inputs]
        ys = self.forward(xs)
        outputs = [Variable(as_array(y)) for y in ys]

        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        # 出力が複数ならoutputs（リスト）をそのまま返す、出力が一つだけならリストの最初の要素を返す(出力先で指定しなくていい)
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

In [9]:
class Add(Function):
    def forward(self, xs):
        x0, x1 = xs
        y = x0 + x1
        return (y,) # この後ループで回すため要素が一つでもタプルにする必要がある(yだけを返すとエラーになる)
    # 関数の出力がタプルなのは不自然（12.2で改善）

In [10]:
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
f = Add()
y = f(x0, x1) # 変長引数のお陰でリストではなく任意の数の引数を代入するだけでいい
print(y.data)

5


12.2 ２つ目の改善：関数を実装しやすく（関数の出力をタプルでなくてもいいようにする＝関数の外でタプルにする）

In [11]:
class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs) # xsリストから全要素を取り出して関数に代入（関数によってysがタプルでない場合がある e.g. 改良型Add関数（下記））
        if not isinstance(ys, tuple): # 出力の要素がタプルでない（要素が一つだけである）場合
            ys = (ys,) # タプルに変換（次の行でループをするため）：
        outputs = [Variable(as_array(y)) for y in ys] # タプル内をループして新しいリストに格納

        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

In [12]:
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y # タプル(y,)で返さなくていい（この後Functionのcall内でタプルに変換してくれる）

12.3 add関数の実装

In [13]:
def add(x0, x1):
    return Add()(x0, x1)

# def add(x0, x1)
#     f = Add()
#     return f(x0, x1)

In [14]:
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
y = add(x0, x1)
print(y.data)

5


step13: 可変長の引数(逆伝播編)

13.1 可変長引数に対応したAddクラスの逆伝播

In [15]:
class Add(Function):

    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy


13.2 Variableクラスの修正(複数の変数に対応)

In [16]:
class Variable:

    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            # x, y = f.input, f.output
            # x.grad = f.backward(y.grad)
            # 渡される微分が複数あることを想定する
            gys = [output.grad for output in f.outputs] # 関数の出力Variableインスタンスが保持する微分を新しいリストに格納
            gxs = f.backward(*gys) # リストを展開して微分を渡して逆伝播計算
            if not isinstance(gxs, tuple): # タプルでない(単なる出力結果が返される)場合
                gxs = (gxs, ) # タプルにする（要素が一つの場合でも次の行でループできる）

            for x, gx in zip(f.inputs, gxs): # i番目の入力xとgxが対応する
                x.grad = gx # Variableインスタンスのxにおける微分データを格納
                if x.creator is not None: # その変数に親がいる場合
                    funcs.append(x.creator) # その親を関数リストに追加してもう１回ループ

13.3 Squareクラスの対応

In [17]:
class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs) # xsリストから全要素を取り出して関数に代入（関数によってysがタプルでない場合がある e.g. 改良型Add関数（下記））
        if not isinstance(ys, tuple): # 出力の要素がタプルでない（関数が出力だけを返す：関数としてより自然）場合
            ys = (ys,) # タプルに変換（次の行でループをするため）：
        outputs = [Variable(as_array(y)) for y in ys] # タプル内をループして新しいリストに格納

        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

In [18]:
class Square(Function):

    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        # x = self.input.data
        x = self.inputs[0].data # *inputs（変長引数）で受け取ると要素が１つでもタプルで受け取るため、計算する際に指定する必要がある
        gx = 2 * x * gy
        return gx

def test(*inputs):
    print(inputs*2) # タプルのまま計算（要素数が２倍に）
    print(inputs[0]*2) # タプルから指定して要素を取り出して計算
    return

test(23)

(23, 23)
46


In [19]:
def square(x):
    return Square() (x)

In [20]:
x = Variable(np.array(2.0))
y = Variable(np.array(3.0))
# a = Variable(x.data + y.data) # 0次元の演算はスカラー（float)になってしまうから改めてnp.arrayに変換する必要がある
# print(a.data, type(a.data))
a = add(x,y) # add関数を通せば、call内で出力がスカラーになってもnp.arrayに変換してかつVariableインスタンスのデータに格納して返してくれる
print(a.data, type(a.data))

b = square(y) # 同様にcall内で出力がスカラーになってもnp.arrayに変換してかつVariableインスタンスのデータに格納して返してくれる
print(b.data, type(b.data))

z = add(square(x), square(y))
z.backward()
print(z.data)
print(x.grad) # x以外の変数を定数と考えてx=2.0とした時の微分(x=2.0に微小の変化与えた時の、zにおける変化量（x=2.0の接線）)
print(y.grad) # y以外の変数を定数と考えてy=3.0とした時の微分(y=3.0に微小の変化与えた時の、zにおける変化量（y=3.0の接線）)

5.0 <class 'numpy.ndarray'>
9.0 <class 'numpy.ndarray'>
13.0
4.0
6.0


step14: 同じ変数を繰り返す

14.2 同じ微分は上書きでなく加算する　＆　微分のリセット（同じ変数で別の計算を行う時）

In [21]:
class Variable:

    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs, )

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None: # 初めて扱われる微分の場合
                    x.grad = gx # 微分を代入
                else: # 同一の微分の場合（同じ変数が入力された場合）
                    x.grad = x.grad + gx # 更新ではなく加算する
                    # x.grad += gxを使うと、渡された微分(gx)がx.gradと同じオブジェクトIDを持ち（x.gradに上書きされ）、結果gxがx.gradと同じ値になってしまう


                if x.creator is not None:
                    funcs.append(x.creator)

    def cleargrad(self): # 別の計算をする時このメソッドを呼ぶ
        self.grad = None

step15, 16: 複雑な計算グラフの対応

16.1 世代の追加

In [22]:
class Variable:

    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0 #親がいない最初の入力は第０世代

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1 # 親(関数)から生み出されたからその出力変数の世代は親より１つ先(親がいない最初の入力は第０世代)

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs, )

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    funcs.append(x.creator)

    def cleargrad(self):
        self.grad = None

In [23]:
class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        # 関数の世代はその入力の世代に合わせる（0世代：入力＋関数、1世代：入力(前の関数の出力)+関数...）
        self.generation = max([x.generation for x in inputs]) # 入力変数が複数ある時、世代が大きい方を採用（ある出力がある関数をスキップして合流することがある）

        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

16.2, 16.3 世代順に取り出すVariableクラスののbackward

In [24]:
class Square(Function):

    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.inputs[0].data # *inputs（変長引数）で受け取ると要素が１つでもタプルで受け取るため、計算する際に指定する必要がある
        gx = 2 * x * gy
        return gx # 演算後にスカラ値になる

In [25]:
class Add(Function):

    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy # Addはそのまま流すから出力の微分gy(np.array)を受け取った場合, np.arrayの型のままである（スカラ値を受け取ればスカラ値をそのまま流す）

In [26]:
class Variable:

    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0 #親がいない最初の入力は第０世代

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1 # 親(関数)から生み出されたからその出力変数の世代は親より１つ先(親がいない最初の入力は第０世代)

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [] # この時点ではまだリストに追加しない
        seen_set = set() # 重複を許さないリストを作成

        # 関数(親)が今までに使われたかをチェックし
        def add_func(f):
            if f not in seen_set: # 今までに同じ関数が使われていない場合
                # print('First time to see in', f, f.generation, 'generation')
                funcs.append(f) # 関数リストに追加
                seen_set.add(f) # 次回以降同じ関数は弾かれる
                funcs.sort(key=lambda x: x.generation) # 関数リスト内を世代順にソート
                # リスト内の関数の世代が[2]→[1,1]→[1,0]になるがソートされて[0,1]になる
                # [2]→[1,1]→[0,1]→１世代の逆伝播が２回あるため、２回１世代の親が呼ばれてseen_setで弾かないと関数リストが[0,0]になり２回同一の計算が行われしまう
            # else:
            #     print(f,'has been already used in ', f.generation, 'generation')

        add_func(self.creator) # 最初(一番出力側)の関数だけ手動で代入（それ以降はループ内のadd_funcsで繰り返す）

        while funcs:
            # print(funcs)
            f = funcs.pop() # 世代順にソートされたfuncsリストの末尾を取り出していく
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple): # Addの逆伝播はタプルで返される
                gxs = (gxs, )

            for x, gx in zip(f.inputs, gxs): # ２つの入力(関数addなど)がある場合,
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator) # その２つの入力に対する２つの親（関数）がadd_func()を通してfuncsリストに追加される）

    def cleargrad(self):
        self.grad = None

In [27]:
x = Variable(np.array(2.0))
a = square(x)
b = square(a)
c = square(a)
y = add(b, c)
y.backward()

print(y.data)
print(c.grad)
print(b.grad)
print(a.grad)
print(x.grad)

32.0
1.0
1.0
16.0
64.0


step17: メモリ管理

17.4 weakrefモジュール（循環参照→弱参照）: GCで循環参照は上手く処理されるがトータルのメモリを抑えるには循環参照を作らないのがベスト

In [28]:
import weakref

In [29]:

class Function:
    def __call__(self, *inputs):
        self.generation = max([x.generation for x in inputs])
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        # self.outputs = outputs # 循環参照を引き起こす（親から子、子から親が参照し合っている）
        self.outputs = [weakref.ref(output) for output in outputs] # 親から子に対して、参照カウントを増やさずに参照（弱参照）
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

In [30]:
class Variable:

    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            # gys = [output.grad for output in f.outputs] # 弱参照にしたため親から子を直接参照できない
            gys = [output().grad for output in f.outputs] # 弱参照のデータにアクセスするには（）を加える
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs, )

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator)

    def cleargrad(self):
        self.grad = None

In [31]:
# 循環参照有りと無しにおけるメモリ使用量のグラフを参照（Memory Usage Circular reference.pngとMemory Usage Non-Circular reference.png）

step18: メモリ使用量を減らすモード

18.1 不要な微分は保持しない

In [32]:

class Function:
    def __call__(self, *inputs):
        self.generation = max([x.generation for x in inputs])
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs # 循環参照を引き起こす（親から子、子から親が参照し合っている）# Jupyterではこれでやる
        # self.outputs = [weakref.ref(output) for output in outputs] # 親から子に対して、参照カウントを増やさずに参照（弱参照）# Jupyterではエラーになるからpythonファイルで実行
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

In [33]:
class Variable:

    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def backward(self, retain_grad=False): # 微分を保持するかの設定（デフォルトは保持しない）
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]
            # gys = [output().grad for output in f.outputs] # 弱参照のデータにアクセスするには（）を加える
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs, )

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator)

            # デフォルトのままなら微分は消去する
            if not retain_grad:
                for y in f.outputs: # 関数の出力に対する微分消去（＝最初の入力における微分だけ保持される）
                    y.grad = None
                    # y().grad = None # f.outputsの要素は弱参照データアクセスのため()を加える

    def cleargrad(self):
        self.grad = None

In [34]:
x = Variable(np.array(2.0))
a = square(x)
b = square(a)
c = square(a)
y = add(b, c)
y.backward()

print(c.grad)
print(b.grad)
print(a.grad)
print(x.grad)

None
None
None
64.0


18.2, 18.3, 18.4 Configクラスによる切り替え

In [35]:
class Config:
    enable_backprop = True # 逆伝播をするかしないかの設定

In [36]:

class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        if Config.enable_backprop: # 設定がオンなら逆伝播のために世代設定、親と子の関連付け、メンバ変数に入力と出力をの保持を行う
            self.generation = max([x.generation for x in inputs])

            for output in outputs:
                output.set_creator(self)
            self.inputs = inputs
            self.outputs = outputs # 循環参照を引き起こす（親から子、子から親が参照し合っている）# Jupyterではこれでやる
            # self.outputs = [weakref.ref(output) for output in outputs] # 親から子に対して、参照カウントを増やさずに参照（弱参照）# Jupyterではエラーになるからpythonファイルで実行
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

In [37]:
class Square(Function):

    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.inputs[0].data # *inputs（変長引数）で受け取ると要素が１つでもタプルで受け取るため、計算する際に指定する必要がある
        gx = 2 * x * gy
        return gx # 演算後にスカラ値になる

In [38]:
class Add(Function):

    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy # Addはそのまま流すから出力の微分gy(np.array)を受け取った場合, np.arrayの型のままである（スカラ値を受け取ればスカラ値をそのまま流す）

In [39]:
Config.enable_backprop = False
x = Variable(np.ones((100, 100, 100)))
y = square(square(square(x)))
print(y)

Config.enable_backprop = True
x = Variable(np.ones((100, 100, 100)))
y = square(square(square(x)))
y.backward

<bound method Variable.backward of <__main__.Variable object at 0x00000200D0AF3FE0>>

18.5 with文によるConfigの自動切り替え（前処理（yield前）＋with文内処理＋(yield後)後処理）

In [40]:
import contextlib

@contextlib.contextmanager # with文内のメイン処理(前伝播)に対する前処理(一時的に逆伝播を無効化)と後処理(デフォルトに戻す)を定義する
def using_config(name, value):
    old_value = getattr(Config, name) #　Configクラスのname(enable_backprop)におけるデフォルト設定(Trueを想定)を取得
    setattr(Config, name, value) # Configクラスにおけるname(enable_backprop)をvalue(False)にセット（逆伝播しない）
    try: # エラー発生の可能性があるコード
        yield # メイン処理（with文内：前伝播）に移動
    finally: # エラーの有無に関係なく必ず実行するコード
        # メイン処理が終わったらここから再開
        setattr(Config, name, old_value) # Configクラスにおけるname(enable_backprop)をvalue(True)に戻す（逆伝播する）

In [41]:
# 逆伝播のためのデータ（世代や入出力データ）を保持しない：前伝播の計算だけする＝推論
# 前伝播が終わると逆伝播のデータ保持設定（デフォルト）に戻る
with using_config("enable_backprop", False):
    x = Variable(np.array(2.0))
    y = square(x)

In [42]:
def no_grad(): # using_config("enable_backprop", False)を簡略化
    return using_config("enable_backprop", False)

In [43]:
with no_grad():
    x = Variable(np.array(2.0))
    y = square(x)

step19: 変数を使いやすく

19.1, 19.2, 19.3, 19.4 変数に名前をつける+Variableインスタンスをndarrayインスタンス(data)のように見せかける

In [44]:
class Variable:

    def __init__(self, data, name=None):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.name = name # 変数に名前を付加
        self.grad = None
        self.creator = None
        self.generation = 0

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def backward(self, retain_grad=False):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]
            # gys = [output().grad for output in f.outputs] # 弱参照のデータにアクセスするには（）を加える
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs, )

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator)

            if not retain_grad:
                for y in f.outputs:
                    y.grad = None
                    # y().grad = None # f.outputsの要素は弱参照データアクセスのため()を加える

    def cleargrad(self):
        self.grad = None

    # メソッドをインスタンス変数(属性)としてアクセスできる(メソッドを呼ぶ時()が必要なくなる＝まるでクラスインスタンスの属性を参照する感じになる)
    @property
    def shape(self):
        return self.data.shape

    @property
    def ndim(self):
        return self.data.ndim

    @property
    def size(self):
        return self.data.size

    @property
    def dtype(self):
        return self.data

    # len() をクラス(Variable)インスタンスに対して呼び出されたときにどのように振る舞うかを定義
    def __len__(self):
        return len(self.data) # Variableクラス内データの要素数を返す

    # print()をクラス(Variable)インスタンスに対して呼び出されたときにどのように振る舞うかを定義
    def __repr__(self):
        if self.data is None:
            return 'variable(None)'
        # データを文字列に変換、かつ改行文字を改行文字+9文字分の空白に置き換えてpに格納
        p = str(self.data).replace('\n', '\n' + ' ' * 9) # 'variable('の９文字分ズラして整形（自動で１文字インデントが入る）
        return 'variable(' + p + ')'

In [45]:
x = Variable(np.array([[1,2,3], [4,5,6], [7,8,9]]))
print(x.shape) # x.data.shapeではなくx.shapeでアクセスできる＝Variableをdataみたいに扱える(いちいち.dataまで書かなくて良くなる)
print(x.ndim)
print(x.size)
print(x.dtype)

print(len(x)) # len(data.x)と書かず直接Variableクラスからアクセスできる

print(x) # print(data.x)と書かず直接Variableクラスからアクセスできる

(3, 3)
2
9
[[1 2 3]
 [4 5 6]
 [7 8 9]]
3
variable([[1 2 3]
          [4 5 6]
          [7 8 9]])


step20: 演算子のオーバーロード(1)

20.1 Mulクラスの実装

In [46]:
class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        if Config.enable_backprop: # 設定がオンなら逆伝播のために世代設定、親と子の関連付け、メンバ変数に入力と出力をの保持を行う
            self.generation = max([x.generation for x in inputs])

            for output in outputs:
                output.set_creator(self)
            self.inputs = inputs
            self.outputs = outputs # 循環参照を引き起こす（親から子、子から親が参照し合っている）# Jupyterではこれでやる
            # self.outputs = [weakref.ref(output) for output in outputs] # 親から子に対して、参照カウントを増やさずに参照（弱参照）# Jupyterではエラーになるからpythonファイルで実行
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

In [47]:
class Mul(Function):
    def forward(self, x0, x1):
        y = x0 * x1
        return y

    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        return gy * x1, gy * x0

In [48]:
def mul(x0, x1):
    return Mul()(x0, x1)

20.2 演算子のオーバーロード(+や*を使えるようにするパート１：左校が特殊メソッドを呼んだ場合)

In [49]:
class Variable:

    def __init__(self, data, name=None):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.name = name
        self.grad = None
        self.creator = None
        self.generation = 0

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def backward(self, retain_grad=False):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]
            # gys = [output().grad for output in f.outputs] # 弱参照のデータにアクセスするには（）を加える
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs, )

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator)

            if not retain_grad:
                for y in f.outputs:
                    y.grad = None
                    # y().grad = None # f.outputsの要素は弱参照データアクセスのため()を加える

    def cleargrad(self):
        self.grad = None

    @property
    def shape(self):
        return self.data.shape

    @property
    def ndim(self):
        return self.data.ndim

    @property
    def size(self):
        return self.data.size

    @property
    def dtype(self):
        return self.data

    def __len__(self):
        return len(self.data)

    def __repr__(self):
        if self.data is None:
            return 'variable(None)'
        p = str(self.data).replace('\n', '\n' + ' ' * 9)
        return 'variable(' + p + ')'

    # ＊が左項クラスの特殊メソッドを呼ぶ（__mul__が定義されていれば）
    def __mul__(self, other): # a*bの場合、a(*の左)がself, b(*の右が)がotherとして引数に渡される
        return mul(self, other) # mul関数を呼んで計算

    # +が左項クラスの特殊メソッドを呼ぶ（__add__が定義されていれば）
    def __add__(self, other): # a+bの場合、a(+の左)がself, b(+の右が)がotherとして引数に渡される
        return add(self, other) # add関数を呼んで計算

    # 以下のように書いても同じ
    # Variable.__mul__ = mul # *に対する振る舞いを関数mul()にする
    # Variable.__add__ = add # +に対する振る舞いを関数add()にする

In [50]:
a = Variable(np.array(3.0))
b = Variable(np.array(2.0))
c = Variable(np.array(1.0))

y = a * b + c
y.backward()

print(y)
print(a.grad)
print(b.grad)
print(c.grad)

variable(7.0)
2.0
3.0
1.0


step21: 演算子のオーバーロード(2)

21.1 ndarrayをそのまま計算に使えるようにする(関数内で自動でndarrayをVariableインスタンスのデータに格納させる)

In [51]:
def as_variable(obj):
    if isinstance(obj, Variable):
        return obj
    return Variable(obj)

In [52]:
class Function:
    def __call__(self, *inputs):
        inputs = [as_variable(x) for x in inputs] # ndarrayを直接関数に入力してもVariableインスタンスに変換する
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        if Config.enable_backprop:
            self.generation = max([x.generation for x in inputs])

            for output in outputs:
                output.set_creator(self)
            self.inputs = inputs
            self.outputs = outputs # 循環参照を引き起こす（親から子、子から親が参照し合っている）# Jupyterではこれでやる
            # self.outputs = [weakref.ref(output) for output in outputs] # 親から子に対して、参照カウントを増やさずに参照（弱参照）# Jupyterではエラーになるからpythonファイルで実行
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

In [53]:
class Mul(Function):
    def forward(self, x0, x1):
        y = x0 * x1
        return y

    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        return gy * x1, gy * x0

In [54]:
class Add(Function):

    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy # Addはそのまま流すから出力の微分gy(np.array)を受け取った場合, np.arrayの型のままである（スカラ値を受け取ればスカラ値をそのまま流す）

In [55]:
x = Variable(np.array(2.0))
# +がVariableインスタンスに作用してadd関数が呼ばれ+の左をself,右をotherとして入力される
y = x + np.array(3.0) # 関数内でndarrayをVariableインスタンスに変換してから計算される
print(y)

variable(5.0)


21.2, 21.3 floatやintと一緒に使えるようにする(+や*を使えるようにするパート2：右項が特殊メソッドを呼んだ場合)

In [56]:
class Variable:

    def __init__(self, data, name=None):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.name = name
        self.grad = None
        self.creator = None
        self.generation = 0

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def backward(self, retain_grad=False):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]
            # gys = [output().grad for output in f.outputs] # 弱参照のデータにアクセスするには（）を加える
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs, )

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator)

            if not retain_grad:
                for y in f.outputs:
                    y.grad = None
                    # y().grad = None # f.outputsの要素は弱参照データアクセスのため()を加える

    def cleargrad(self):
        self.grad = None

    @property
    def shape(self):
        return self.data.shape

    @property
    def ndim(self):
        return self.data.ndim

    @property
    def size(self):
        return self.data.size

    @property
    def dtype(self):
        return self.data

    def __len__(self):
        return len(self.data)

    def __repr__(self):
        if self.data is None:
            return 'variable(None)'
        p = str(self.data).replace('\n', '\n' + ' ' * 9)
        return 'variable(' + p + ')'

    def __mul__(self, other):
        return mul(self, other)

    def __add__(self, other):
        return add(self, other)

    # ＊の左項クラスの特殊メソッドが定義されていなかった場合、右項の特殊メソッドが呼ばれる
    def __rmul__(self, other): # *の右項がself, 左項がotherが引数として渡される
        return mul(self, other) # 関数mul()の処理

    # ＊の左項クラスの特殊メソッドが定義されていなかった場合、右項の特殊メソッドが呼ばれる
    def __radd__(self, other):  # *の右項がself, 左項がotherが引数として渡される
        return add(self, other) # 関数add()の処理

    # 以下のように書いても同じ
    # Variable.__rmul__ = mul # *に対する振る舞いを関数mul()にする
    # Variable.__radd__ = add # +に対する振る舞いを関数add()にする

In [57]:
# Variableクラス*floatクラスなら__mul__が呼ばれ、そのままの順番で関数に渡されて、floatクラスがndarrayに変換される
# floatクラス*Variableクラスなら__rmul__が呼ばれ、順番を逆にして関数に渡されて、結局同じくfloatクラスがndarrayに変換される
def mul(x0, x1): # したがって、other(x1)には必ず非Variableクラスが来る(特殊メソッドを呼んだクラスがself,もう一方がotherになる)
    x1 = as_array(x1) # floatがndarray変換
    return Mul()(x0, x1) # Functionクラス内でndarrayがVariabelインスタンスのデータに格納される

In [58]:
# Variableクラス+floatクラスなら__add__が呼ばれ、そのままの順番で関数に渡されて、floatクラスがndarrayに変換される
# floatクラス+Variableクラスなら__radd__が呼ばれ、順番を逆にして関数に渡されて、結局同じくfloatクラスがndarrayに変換される
def add(x0, x1): # したがって、other(x1)には必ず非Variableクラスが来る(特殊メソッドを呼んだクラスがself,もう一方がotherになる)
    x1 = as_array(x1) # floatがndarray変換
    return Add()(x0, x1) # Functionクラス内でndarrayがVariabelインスタンスのデータに格納される

In [59]:
x = Variable(np.array(2.0))
y = 3.0 * x + 1.0
print(y)

variable(7.0)


21.4 左項がndarrayである場合

In [60]:
x = Variable(np.array(2.0))
y1 = x + np.array([2.0])
y2 = np.array([2.0]) + x # ndarrayの特殊メソッド(__add__)が呼ばれて出力形式が異なってしまう
print(y1)
print(y2)

variable([4.])
[variable(4.0)]


In [61]:
class Variable:
    __array_priority__ = 200 # 特殊メソッドを呼び出す優先度（左項にndarrayインスタンスがあってもVariableインスタンスの特殊メソッドが優先される）

    def __init__(self, data, name=None):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.name = name
        self.grad = None
        self.creator = None
        self.generation = 0

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def backward(self, retain_grad=False):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]
            # gys = [output().grad for output in f.outputs] # 弱参照のデータにアクセスするには（）を加える
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs, )

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator)

            if not retain_grad:
                for y in f.outputs:
                    y.grad = None
                    # y().grad = None # f.outputsの要素は弱参照データアクセスのため()を加える

    def cleargrad(self):
        self.grad = None

    @property
    def shape(self):
        return self.data.shape

    @property
    def ndim(self):
        return self.data.ndim

    @property
    def size(self):
        return self.data.size

    @property
    def dtype(self):
        return self.data

    def __len__(self):
        return len(self.data)

    def __repr__(self):
        if self.data is None:
            return 'variable(None)'
        p = str(self.data).replace('\n', '\n' + ' ' * 9)
        return 'variable(' + p + ')'

    def __mul__(self, other):
        return mul(self, other)

    def __add__(self, other):
        return add(self, other)

    def __rmul__(self, other):
        return mul(self, other)

    def __radd__(self, other):
        return add(self, other)

In [62]:
x = Variable(np.array(2.0))
y1 = x + np.array([2.0])
y2 = np.array([2.0]) + x # Variableインスタンスの特殊メソッドが優先され、出力形式が統一される
print(y1)
print(y2)

variable([4.])
variable([4.])


step22: 演算子のオーバーロード（３）

22.1 負数

In [63]:
class  Neg(Function):
    def forward(self, x):
        return-x

    def backward(self, gy):
        return -gy

In [64]:
def neg(x):
    return Neg()(x)

22.2 引き算

In [65]:
class Sub(Function):
    def forward(self, x0, x1):
        y = x0 - x1
        return y

    def backward(self, gy):
        return gy, -gy

In [66]:
# 右項がスカラー値の場合,そのままx1に渡されndarrayに変換される
def sub(x0, x1):
    x1 = as_array(x1)
    return Sub()(x0, x1)

In [67]:
# 左項がスカラー値の場合,左項がother(x1)に渡されndarrayに変換されるが、左右の項が逆になってしまう（そのまま計算すると結果が異なってしまう）
def rsub(x0, x1):
    x1 = as_array(x1)
    return Sub()(x1, x0) # 元の項の並びに戻して引き算を行う（足し算や掛け算はその必要がない）

22.3 割り算

In [68]:
class Div(Function):
    def forward(self, x0, x1):
        y = x0 / x1
        return y

    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        gx0 = gy /x1
        gx1 = gy * (-x0 / x1 ** 2)
        return gx0, gx1

In [69]:
# 右項がスカラー値の場合,そのままx1に渡されndarrayに変換される
def div(x0, x1):
    x1 = as_array(x1)
    return Div()(x0, x1)

In [70]:
# 左項がスカラー値の場合,左項がother(x1)に渡されndarrayに変換されるが、左右の項が逆になってしまう（そのまま計算すると結果が異なってしまう）
def rdiv(x0, x1):
    x1 = as_array(x1)
    return Div()(x1,x0) # 元の項の並びに戻して割り算を行う（足し算や掛け算はその必要がない）

22.4 累乗

In [71]:
class Pow(Function):
    def __init__(self, c):
        self.c = c

    def forward(self, x):
        y = x ** self.c
        return y

    def backward(self, gy):
        x = self.inputs[0].data
        c = self.c
        gx = c * x ** (c - 1) * gy
        return gx

In [72]:
def pow(x, c):
    return Pow(c)(x)

22.5 自由な四則演算

In [73]:
class Variable:
    __array_priority__ = 200

    def __init__(self, data, name=None):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.name = name
        self.grad = None
        self.creator = None
        self.generation = 0

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def backward(self, retain_grad=False):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]
            # gys = [output().grad for output in f.outputs] # 弱参照のデータにアクセスするには（）を加える
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs, )

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator)

            if not retain_grad:
                for y in f.outputs:
                    y.grad = None
                    # y().grad = None # f.outputsの要素は弱参照データアクセスのため()を加える

    def cleargrad(self):
        self.grad = None

    @property
    def shape(self):
        return self.data.shape

    @property
    def ndim(self):
        return self.data.ndim

    @property
    def size(self):
        return self.data.size

    @property
    def dtype(self):
        return self.data

    def __len__(self):
        return len(self.data)

    def __repr__(self):
        if self.data is None:
            return 'variable(None)'
        p = str(self.data).replace('\n', '\n' + ' ' * 9)
        return 'variable(' + p + ')'

    def __mul__(self, other):
        return mul(self, other)

    def __add__(self, other):
        return add(self, other)

    def __rmul__(self, other):
        return mul(self, other)

    def __radd__(self, other):
        return add(self, other)

    # 負数
    def __neg__(self):
        return neg(self)

    # 引き算
    def __sub__(self, other):
        return sub(self, other)

    def __rsub__(self, other):
        return sub(self, other)

    # 割り算
    def __truediv__(self, other):
        return div(self, other)

    def __rtruediv__(self, other):
        return rdiv(self, other)

    # 累乗
    def __pow__(self, other):
        return pow(self, other)

In [74]:
x = Variable(np.array(2.0))
y = -x
print(y)

y1 = x - 2.0
y2 = 2.0 -x
print(y1)
print(y2)

y3 = x / 2.0
y4 = 2.0 / x
print(y3)
print(y4)

y5 = x**3
print(y5)

variable(-2.0)
variable(0.0)
variable(0.0)
variable(1.0)
variable(1.0)
variable(8.0)


step23: パッケージとしてまとめる

23.1, 23.2, 23.3　step22まで実装したコア機能を１つのファイルにまとめてdezeroディレクトリ（フォルダ）に収納（パッケージ化）

In [75]:
# パッケージからVariableクラスを呼び出す（もういちいちVariableクラスを定義しなくていい）
import numpy as np
import sys
sys.path.append(r'C:\Users\jiwon\OneDrive\Desktop\DL3')
from dezero.core_simple import Variable

x =Variable(np.array(1.0))
print(x)

variable(1.0)


23.4 __init__.pyをdezeroに搭載することで、dezeroを呼び出した沖core_simpleを自動で呼び出す

In [76]:
import numpy as np
import sys
sys.path.append(r'C:\Users\jiwon\OneDrive\Desktop\DL3')
from dezero import Variable #dezero.core_simpleまで書かなくてもその機能が呼び出されう

x =Variable(np.array(1.0))
print(x)

variable(1.0)


23.5 dezeroのインポート（環境によって親ディレクトリをパスに追加）

In [77]:
# このスクリプトがpythonファイル上のものか(pythonファイルならグローバル名前空間に__file__が定義されている)か確認
if '__file__' in globals(): # Jupyter Notebookやインタラクティブシェルで実行するとFalseになる
    import os, sys
    sys.path.append(os.path.join(os.path.dirname(__file__), '..')) # 実行ファイルのディレクトリに対する親ディレクトリ(ここではDL3)をパスに追加
    # sys.path.append(r'C:\Users\jiwon\OneDrive\Desktop\DL3') と実質同じ

step24 複雑な関数の微分

24.1 Sphere関数

In [78]:
def sphere(x, y):
    z = x ** 2 + y ** 2
    return z

In [79]:
import numpy as np
import sys
sys.path.append(r'C:\Users\jiwon\OneDrive\Desktop\DL3')
from dezero import Variable

x = Variable(np.array(1.0))
y = Variable(np.array(1.0))
z = sphere(x, y)
z.backward()
print(x.grad, y.grad)

2.0 2.0


24.2 matyas関数

In [80]:
def matyas(x, y):
    z = 0.26 * (x ** 2 + y ** 2) - 0.48 * x * y
    return z

In [81]:
x = Variable(np.array(1.0))
y = Variable(np.array(1.0))
z = matyas(x, y)
z.backward()
print(x.grad, y.grad)

0.040000000000000036 0.040000000000000036


24.3 Goldstein-Price関数

In [82]:
def goldstein(x, y):
    z = (1 + (x + y + 1)**2 * (19 - 14*x + 3*x**2 - 14*y + 6*x*y + 3*y**2)) * \
        (30 + (2*x - 3*y)**2 * (18 - 32*x + 12*x**2 + 48*y - 36*x*y + 27*y**2))
    return z

In [83]:
x = Variable(np.array(1.0))
y = Variable(np.array(1.0))
z = goldstein(x, y)
z.backward()
print(x.grad, y.grad)

-5376.0 8064.0
